In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from unidecode import unidecode

# Carregue os dados do CSV em um DataFrame do Pandas:
df = pd.read_csv('./dados.csv', delimiter=';')

# Fazer amostragem aleatória de 20.000 linhas e remover linhas com valores ausentes:
df = df.sample(n=30000).dropna()

# Divida os dados em recursos e rótulos:
X = df['Name']
y = df['Gender']

# Remova acentos e caracteres problemáticos:


def remove_accents(text):
    return unidecode(text)


# Transforme os dados de texto em vetores numéricos usando o CountVectorizer do Scikit-learn:
vectorizer = CountVectorizer(preprocessor=remove_accents)
X = vectorizer.fit_transform(X)

# Divida os dados em conjuntos de treinamento e teste:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Padronize os recursos usando o StandardScaler do Scikit-learn:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.toarray())
X_test = scaler.transform(X_test.toarray())

# Crie um modelo sequencial com o Keras:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile o modelo com a função de perda, otimizador e métricas:
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

# Treine o modelo com os dados de treinamento:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train, y_train, epochs=20, batch_size=32,
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Avalie o desempenho do modelo com os dados de teste:
score = model.evaluate(X_test, y_test, batch_size=32)
print("Acurácia:", score[1])

# Classifique o nome "Afonso":
name_to_classify = vectorizer.transform(["Afonso"])
predicted_gender = model.predict(name_to_classify)
predicted_gender = np.round(predicted_gender)
print("O gênero do nome Afonso é:", predicted_gender)
